## Appendix: The `click` module

### Installation of `click`

The installation of the module is easy:

```bash
pip install -U click
```

Note that
* `install` is the command
* `-U` is an option, a shorthand for `--upgrade` which forces pip to install the latest version of that module
* `click` is the argument, in our case the name of the module

Documentation of the `click` module can be found [here](https://click.palletsprojects.com/en/7.x/).

```python
import click

@click.command()
@click.option("--count",  "-c", default=1, help="Number of greetings.")
@click.option("--polite", "-p", is_flag=True)
@click.option("--name",   "-n", help="The person to greet.")
def hello(count, polite, name):
    if polite:
        greeting = "Your Serene Highness"
    else:
        greeting = "Hello"

    for _ in range(count):
        click.echo(f"{greeting} {name}!")

if __name__ == "__main__":
    hello()
```

The first line we already understand:

```python
import click
```

This just imports the click module, ready to be used.

Next, we discover some new elements right above the function:

```python
@click.command()
@click.option("--count",  "-c", default=1, help="Number of greetings.")
@click.option("--polite", "-p", is_flag=True)
@click.option("--name",   "-n", help="The person to greet.")
def hello(count, polite, name):
```

These elements are so called **decorators**. Their functionality is to **wrap** a function or a method. Before the `def hello()` function is being executed, first the `click.command()` is executed, then three times a `click.option()` decorator, before we actually see the function definition.

The options `--count`, `--polite` and `--name` are passed as parameters `count`, `polite` and `name` to our  function, in the order they appear.

You can also define kebab-style options, e.g. `--test-only` which then will be passed as a `test_only` parameter to your function.

```python
def hello(count, polite, name):
    if polite:
        greeting = "Your Serene Highness"
    else:
        greeting = "Hello"
        
    if not name:
        name = "unknown stranger"

    for x in range(count):
        click.echo(f"{greeting} {name}!")
```

The rest of the code in this method prints out a message a number of times, using the `click.echo()` method and the convenient [f-string notation](https://realpython.com/python-f-strings/)

### Boolean values (flags)

Often we want to adjust the behaviour of our command line tool by providing boolean flags. The presence of the flag will set it to True:

```python

@click.option(
    '-d', '--delete',
    is_flag=True,
    help='delete this user'
)
```

### Passwords

When you enter a password or some other confidential value, you don't want to see on the screen what you write. The `hide_input=True` parameter will prevent this.

When resetting passwords, one often needs to enter the new password twice, in case you entered a typo. Use `confirmation_prompt=True` to show the password prompt a second time and compare the two inputs.

```python
@click.option(
    '--password', 
    prompt=True,                 # does what it says: it prompts
    confirmation_prompt=True,    # the password has to be entered twice
    hide_input=True              # do not show the input characters
)
```

## Arguments

[Arguments](https://click.palletsprojects.com/en/8.0.x/arguments/) work almost like options with the difference that they are *positional*. By adding the parameter `nargs=-1`, you can add as many values for one argument (only works for *one* argument, as it eats up all other arguments):

Example:

```python
@click.command()
@click.argument('src', nargs=-1)
@click.argument('dst', nargs=1)
def copy(src, dst):
    """Move file SRC to DST."""
    for fn in src:
        click.echo(f"move {fn} to folder {dst}")
```

### Exercise 2

modify hello_click.py

- add a help text for the `--polite` option and test it
- add a `prompt` parameter to obtain a name, if it is not provided
- add an *optional* [file argument](https://click.palletsprojects.com/en/7.x/arguments/#file-path-arguments) (with `nargs=-1`)
- if the argument is present, open the content of the file and print it after the greeting

## Sub-Commands

Sub-commands are a way to conveniently group options for a given task. For example, the [ethz-iam-webservice](https://gitlab.ethz.ch/vermeul/ethz-iam-webservice) CLI offers the `person`, `user` and the `group` command. Below you see some examples how the utility takes a **command** followed by an **argument** and some **options**:

```bash
$ iam person swen@ethz.ch
$ iam user vermeul
$ iam user vermeul --grant-service LDAPS
$ iam group my_group
$ iam group my_group -a new_user1 -a new_user2 -r old_user3
$ iam group my_group --delete
```

### How to implement a sub-command

1. define the main entry function, e.g. `def do_something()`
2. decorate it with the `@click.group()` decorator
3. define the subcommand function, e.g. `def sing()`
4. decorate it with the `@do_something.command()` decorator

### How to pass context

From the [documentation](https://click.palletsprojects.com/en/8.0.x/commands/#nested-handling-and-contexts)

```python
@click.group()
@click.option('--debug/--no-debug', default=False)
@click.pass_context
def cli(ctx, debug):
    # ensure that ctx.obj exists and is a dict (in case `cli()` is called
    # by means other than the `if` block below)
    ctx.ensure_object(dict)

    ctx.obj['DEBUG'] = debug

@cli.command()
@click.pass_context
def sync(ctx):
    click.echo(f"Debug is {'on' if ctx.obj['DEBUG'] else 'off'}")

if __name__ == '__main__':
    cli(obj={}`
```

### Exercise 3

Open [click_commands.py](click_commands.py)

- observe how the two sub-commands are defined
- try to pass some options and arguments to the sub-commands
- implement an example which takes a value of the main option and passes it to the sub-command